In [ ]:
#Prophet

In [1]:
!pip install prophet

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
import os
from prophet import Prophet
import joblib
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import matplotlib.pyplot as plt
import warnings

# -----------------------------
# Suppress warnings
# -----------------------------
warnings.filterwarnings("ignore")

# -----------------------------
# Paths
# -----------------------------
input_folder = "dataset"
output_models = "prophet_output_models"
output_csv = "prophet_output_csv"
output_metrics_csv = "prophet_metrics.csv"
output_graphs = "prophet_output_graphs"

os.makedirs(output_models, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)

# -----------------------------
# Prepare metrics storage
# -----------------------------
metrics_list = []

# -----------------------------
# Function to robustly parse dates (all formats)
# -----------------------------
def parse_dates_safe(date_series):
    return pd.to_datetime(date_series, errors='coerce', dayfirst=False, infer_datetime_format=True)

# -----------------------------
# Function to calculate MAPE safely
# -----------------------------
def mean_absolute_percentage_error_safe(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    # Avoid division by zero
    mask = y_true != 0
    if mask.sum() == 0:
        return np.nan
    mape = np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100
    # Handle any remaining inf or NaN
    if np.isnan(mape) or np.isinf(mape):
        return np.nan
    return mape

# -----------------------------
# Process each CSV
# -----------------------------
for file in os.listdir(input_folder):
    if file.endswith(".csv"):
        file_path = os.path.join(input_folder, file)
        print(f"========== Processing: {file} ==========")

        # Load CSV
        df = pd.read_csv(file_path)

        # -----------------------------
        # Convert 'Date' column safely
        # -----------------------------
        df['Date'] = parse_dates_safe(df['Date'])
        df = df.sort_values('Date')

        # Drop rows with NaT after conversion
        df = df.dropna(subset=['Date'])
        if df.empty:
            print(f"⚠️ No valid dates found in {file}. Skipping...")
            continue

        # -----------------------------
        # Handle missing values in Average Price
        # -----------------------------
        if df['Average Price'].isna().sum() > 0:
            mean_value = df['Average Price'].mean()
            df['Average Price'].fillna(mean_value, inplace=True)
            print(f"Filled {df['Average Price'].isna().sum()} missing values with mean: {mean_value:.2f}")

        # Round actual values
        df['Average Price'] = df['Average Price'].round(2)

        # -----------------------------
        # Prepare data for Prophet
        # -----------------------------
        prophet_df = df[['Date', 'Average Price']].rename(columns={'Date': 'ds', 'Average Price': 'y'})

        # Skip files with less than 2 valid rows (Prophet requirement)
        if len(prophet_df) < 2:
            print(f"⚠️ Not enough data points in {file} for Prophet. Skipping...")
            continue

        # Build and fit Prophet model
        model = Prophet(daily_seasonality=True)
        model.fit(prophet_df)

        # Predict for existing dates
        future = model.make_future_dataframe(periods=0)
        forecast = model.predict(future)
        df['Predicted'] = forecast['yhat']  # keep full precision for metrics

        # -----------------------------
        # Handle NaNs in predictions for metrics
        # -----------------------------
        metrics_df = df.dropna(subset=['Average Price', 'Predicted'])
        y_true = metrics_df['Average Price'].values
        y_pred = metrics_df['Predicted'].values

        # Round predictions for saving
        df['Predicted'] = df['Predicted'].round(2)

        # Rename Average Price to Actual in main df for plotting
        df.rename(columns={'Average Price': 'Actual'}, inplace=True)

        # -----------------------------
        # Calculate metrics
        # -----------------------------
        mae = round(mean_absolute_error(y_true, y_pred), 2)
        rmse = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)
        r2 = round(r2_score(y_true, y_pred), 4)
        mape = round(mean_absolute_percentage_error_safe(y_true, y_pred), 2)
        accuracy = round(100 - mape if not np.isnan(mape) else np.nan, 2)

        print(f"Metrics for {file}:")
        print(f"  MAE        : {mae}")
        print(f"  RMSE       : {rmse}")
        print(f"  R²         : {r2}")
        print(f"  MAPE(%)    : {mape}")
        print(f"  Accuracy(%) : {accuracy}\n")

        # Save metrics
        metrics_list.append({
            'File': file,
            'MAE': mae,
            'RMSE': rmse,
            'R2': r2,
            'MAPE(%)': mape,
            'Accuracy(%)': accuracy
        })

        # -----------------------------
        # Save model
        # -----------------------------
        model_file = os.path.join(output_models, file.replace(".csv", "_prophet_model.pkl"))
        joblib.dump(model, model_file)

        # -----------------------------
        # Save CSV with only Date, Actual, and Predicted
        # -----------------------------
        save_df = df[['Date', 'Actual', 'Predicted']]
        updated_csv_path = os.path.join(output_csv, file.replace(".csv", "_prophet_updated.csv"))
        save_df.to_csv(updated_csv_path, index=False)
        print(f"Saved updated CSV with Date, Actual & Predicted: {updated_csv_path}")

        # -----------------------------
        # Add MA7 & MA30 for graphs
        # -----------------------------
        df['MA7'] = df['Actual'].rolling(7).mean()
        df['MA30'] = df['Actual'].rolling(30).mean()

        # -----------------------------
        # Plot graph
        # -----------------------------
        plt.figure(figsize=(12, 6))
        plt.plot(df['Date'], df['Actual'], label='Actual', color='blue')
        plt.plot(df['Date'], df['Predicted'], label='Predicted', color='red')
        plt.plot(df['Date'], df['MA7'], label='MA7', color='green', linestyle='--')
        plt.plot(df['Date'], df['MA30'], label='MA30', color='orange', linestyle='--')
        plt.title(f"{file} - Actual vs Predicted with Moving Averages")
        plt.xlabel("Date")
        plt.ylabel("Price")
        plt.legend()
        plt.grid(True)
        graph_path = os.path.join(output_graphs, file.replace(".csv", "_prophet_graph.png"))
        plt.savefig(graph_path, bbox_inches='tight')
        plt.close()

# -----------------------------
# Save all metrics to CSV
# -----------------------------
metrics_df = pd.DataFrame(metrics_list)
metrics_df.to_csv(output_metrics_csv, index=False)
print(f"✅ Multiple CSVs processed! Metrics saved to {output_metrics_csv}. Models, updated CSVs, and graphs saved in respective folders.")

========== Processing: onion_Bagalkot_daily.csv ==========


18:28:28 - cmdstanpy - INFO - Chain [1] start processing
18:28:42 - cmdstanpy - INFO - Chain [1] done processing


Metrics for onion_Bagalkot_daily.csv:
  MAE        : 642.4
  RMSE       : 848.76
  R²         : -0.3121
  MAPE(%)    : 67.88
  Accuracy(%) : 32.12

Saved updated CSV with Date, Actual & Predicted: prophet_output_csv\onion_Bagalkot_daily_prophet_updated.csv
========== Processing: onion_Bangalore_daily.csv ==========


18:28:45 - cmdstanpy - INFO - Chain [1] start processing
18:29:11 - cmdstanpy - INFO - Chain [1] done processing


Metrics for onion_Bangalore_daily.csv:
  MAE        : 571.46
  RMSE       : 824.89
  R²         : 0.4287
  MAPE(%)    : 46.04
  Accuracy(%) : 53.96

Saved updated CSV with Date, Actual & Predicted: prophet_output_csv\onion_Bangalore_daily_prophet_updated.csv
========== Processing: onion_Belgaum_daily.csv ==========


...

Metrics for onion_Shimoga_daily.csv:
  MAE        : 577.69
  RMSE       : 842.86
  R²         : -0.5081
  MAPE(%)    : 55.01
  Accuracy(%) : 44.99

Saved updated CSV with Date, Actual & Predicted: prophet_output_csv\onion_Shimoga_daily_prophet_updated.csv
========== Processing: onion_Tumkur_daily.csv ==========


18:38:13 - cmdstanpy - INFO - Chain [1] start processing
18:38:26 - cmdstanpy - INFO - Chain [1] done processing


Metrics for onion_Tumkur_daily.csv:
  MAE        : 1198.4
  RMSE       : 2070.52
  R²         : 0.5927
  MAPE(%)    : 35.59
  Accuracy(%) : 64.41

Saved updated CSV with Date, Actual & Predicted: prophet_output_csv\onion_Tumkur_daily_prophet_updated.csv
========== Processing: onion_Udupi_daily.csv ==========


18:38:31 - cmdstanpy - INFO - Chain [1] start processing
18:38:41 - cmdstanpy - INFO - Chain [1] done processing


Metrics for onion_Udupi_daily.csv:
  MAE        : 1024.61
  RMSE       : 1374.78
  R²         : -0.4535
  MAPE(%)    : 71.08
  Accuracy(%) : 28.92

Saved updated CSV with Date, Actual & Predicted: prophet_output_csv\onion_Udupi_daily_prophet_updated.csv
✅ Multiple CSVs processed! Metrics saved to prophet_metrics.csv. Models, updated CSVs, and graphs saved in respective folders.


In [ ]:
#ARIMA

In [3]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib
import warnings

# -----------------------------
# Suppress warnings
# -----------------------------
warnings.filterwarnings("ignore")

# -----------------------------
# Paths
# -----------------------------
input_folder = "dataset"
output_models = "arima_output_models"
output_csv = "arima_output_csv"
output_metrics_csv = "arima_metrics.csv"
output_graphs = "arima_output_graphs"

os.makedirs(output_models, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)

# -----------------------------
# Metrics storage
# -----------------------------
metrics_list = []

# -----------------------------
# Robust date parsing (allow all formats)
# -----------------------------
def parse_dates_safe(date_series):
    return pd.to_datetime(date_series, errors='coerce', infer_datetime_format=True)

# -----------------------------
# Safe MAPE calculation (handle NaN/Inf)
# -----------------------------
def mean_absolute_percentage_error_safe(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mask = y_true != 0
    if mask.sum() == 0:
        return 0.0
    mape = np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100
    if np.isnan(mape) or np.isinf(mape):
        return 0.0
    return mape

# -----------------------------
# Process each CSV
# -----------------------------
for file in os.listdir(input_folder):
    if file.endswith(".csv"):
        file_path = os.path.join(input_folder, file)
        print(f"========== Processing: {file} ==========")

        # Load CSV
        df = pd.read_csv(file_path)

        # -----------------------------
        # Convert 'Date' column safely
        # -----------------------------
        df['Date'] = parse_dates_safe(df['Date'])
        df = df.sort_values('Date')

        # Drop rows with invalid dates
        df = df.dropna(subset=['Date'])
        if df.empty:
            print(f"⚠️ No valid dates found in {file}. Skipping...")
            continue

        # -----------------------------
        # Handle missing Average Price
        # -----------------------------
        if df['Average Price'].isna().sum() > 0:
            mean_value = df['Average Price'].mean()
            df['Average Price'].fillna(mean_value, inplace=True)
            print(f"Filled {df['Average Price'].isna().sum()} missing values with mean: {mean_value:.2f}")

        df['Average Price'] = df['Average Price'].round(2)

        # -----------------------------
        # Fit ARIMA model
        # -----------------------------
        order = (5, 1, 0)
        model = ARIMA(df['Average Price'], order=order)
        model_fit = model.fit()

        # Predict for existing dates
        df['Predicted'] = model_fit.predict(start=0, end=len(df)-1)
        df['Predicted'] = df['Predicted'].round(2)

        # -----------------------------
        # Rename Average Price to Actual
        # -----------------------------
        df.rename(columns={'Average Price': 'Actual'}, inplace=True)

        # -----------------------------
        # Calculate metrics
        # -----------------------------
        y_true = df['Actual'].values
        y_pred = df['Predicted'].values
        mae = round(mean_absolute_error(y_true, y_pred), 2)
        rmse = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)
        r2 = round(r2_score(y_true, y_pred), 4)
        mape = round(mean_absolute_percentage_error_safe(y_true, y_pred), 2)
        accuracy = round(100 - mape, 2)

        print(f"Metrics for {file}:")
        print(f"  MAE        : {mae}")
        print(f"  RMSE       : {rmse}")
        print(f"  R²         : {r2}")
        print(f"  MAPE(%)    : {mape}")
        print(f"  Accuracy(%) : {accuracy}\n")

        # Save metrics
        metrics_list.append({
            'File': file,
            'MAE': mae,
            'RMSE': rmse,
            'R2': r2,
            'MAPE(%)': mape,
            'Accuracy(%)': accuracy
        })

        # -----------------------------
        # Save model
        # -----------------------------
        model_file = os.path.join(output_models, file.replace(".csv", "_arima_model.pkl"))
        joblib.dump(model_fit, model_file)

        # -----------------------------
        # Save CSV with only Date, Actual, Predicted
        # -----------------------------
        save_df = df[['Date', 'Actual', 'Predicted']]
        updated_csv_path = os.path.join(output_csv, file.replace(".csv", "_arima_updated.csv"))
        save_df.to_csv(updated_csv_path, index=False)
        print(f"Saved updated CSV with Date, Actual & Predicted: {updated_csv_path}")

        # -----------------------------
        # Add MA7 & MA30 for graphs
        # -----------------------------
        df['MA7'] = df['Actual'].rolling(7).mean()
        df['MA30'] = df['Actual'].rolling(30).mean()

        # -----------------------------
        # Plot graph
        # -----------------------------
        plt.figure(figsize=(12,6))
        plt.plot(df['Date'], df['Actual'], label='Actual', color='blue')
        plt.plot(df['Date'], df['Predicted'], label='Predicted', color='red')
        plt.plot(df['Date'], df['MA7'], label='MA7', color='green', linestyle='--')
        plt.plot(df['Date'], df['MA30'], label='MA30', color='orange', linestyle='--')
        plt.title(f"{file} - Actual vs Predicted with Moving Averages")
        plt.xlabel("Date")
        plt.ylabel("Price")
        plt.legend()
        plt.grid(True)
        graph_path = os.path.join(output_graphs, file.replace(".csv", "_arima_graph.png"))
        plt.savefig(graph_path, bbox_inches='tight')
        plt.close()

# -----------------------------
# Save all metrics to CSV
# -----------------------------
metrics_df = pd.DataFrame(metrics_list)
metrics_df.to_csv(output_metrics_csv, index=False)
print(f"✅ Multiple CSVs processed! Metrics saved to {output_metrics_csv}. Models, updated CSVs, and graphs saved in respective folders.")

========== Processing: onion_Bagalkot_daily.csv ==========
Metrics for onion_Bagalkot_daily.csv:
  MAE        : 114.62
  RMSE       : 196.27
  R²         : 0.9339
  MAPE(%)    : 11.23
  Accuracy(%) : 88.77

Saved updated CSV with Date, Actual & Predicted: arima_output_csv\onion_Bagalkot_daily_arima_updated.csv
========== Processing: onion_Bangalore_daily.csv ==========
Metrics for onion_Bangalore_daily.csv:
  MAE        : 367.99
  RMSE       : 557.5
  R²         : 0.6998
  MAPE(%)    : 23.55
  Accuracy(%) : 76.45

Saved updated CSV with Date, Actual & Predicted: arima_output_csv\onion_Bangalore_daily_arima_updated.csv
========== Processing: onion_Belgaum_daily.csv ==========
Metrics for onion_Belgaum_daily.csv:
  MAE        : 118.47
  RMSE       : 232.88
  R²         : 0.9444
  MAPE(%)    : 11.35
  Accuracy(%) : 88.65

Saved updated CSV with Date, Actual & Predicted: arima_output_csv\onion_Belgaum_daily_arima_updated.csv
========== Processing: onion_Bellary_daily.csv ==========
Metrics

In [ ]:
#SARIMAX

In [4]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings
import joblib

warnings.filterwarnings("ignore")  # Ignore warnings

# -----------------------------
# Paths
# -----------------------------
input_folder = "dataset"
output_models = "sarimax_output_models"
output_csv = "sarimax_output_csv"
output_graphs = "sarimax_output_graphs"
output_metrics_csv = "sarimax_metrics.csv"

os.makedirs(output_models, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)

# -----------------------------
# Metrics storage
# -----------------------------
metrics_list = []

# -----------------------------
# Robust date parsing (allow all formats)
# -----------------------------
def parse_dates_safe(date_series):
    return pd.to_datetime(date_series, errors='coerce', infer_datetime_format=True)

# -----------------------------
# Safe MAPE calculation (handle NaN/Inf)
# -----------------------------
def mean_absolute_percentage_error_safe(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mask = y_true != 0
    if mask.sum() == 0:
        return 0.0
    mape = np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100
    if np.isnan(mape) or np.isinf(mape):
        return 0.0
    return mape

# -----------------------------
# Process each CSV
# -----------------------------
for file in os.listdir(input_folder):
    if file.endswith(".csv"):
        file_path = os.path.join(input_folder, file)
        print(f"========== Processing: {file} ==========")

        # Load CSV
        df = pd.read_csv(file_path)

        # -----------------------------
        # Convert 'Date' column safely
        # -----------------------------
        df['Date'] = parse_dates_safe(df['Date'])
        df = df.sort_values('Date')
        df.set_index('Date', inplace=True)

        # Drop rows with invalid dates
        df = df.dropna(subset=['Average Price'])
        if df.empty:
            print(f"⚠️ No valid data found in {file}. Skipping...")
            continue

        # Handle missing values
        if df['Average Price'].isna().sum() > 0:
            mean_value = df['Average Price'].mean()
            df['Average Price'].fillna(mean_value, inplace=True)
            print(f"Filled {df['Average Price'].isna().sum()} missing values with mean: {mean_value:.2f}")

        # Round actual values
        df['Average Price'] = df['Average Price'].round(2)

        # Add moving averages
        df['MA_7'] = df['Average Price'].rolling(window=7).mean()
        df['MA_30'] = df['Average Price'].rolling(window=30).mean()

        # SARIMAX order (tune if needed)
        order = (1, 1, 1)
        seasonal_order = (1, 1, 1, 7)

        # Fit SARIMAX model
        model = SARIMAX(df['Average Price'],
                        order=order,
                        seasonal_order=seasonal_order,
                        enforce_stationarity=False,
                        enforce_invertibility=False)
        model_fit = model.fit(disp=False)

        # Save model
        model_file = os.path.join(output_models, file.replace(".csv", "_sarimax_model.pkl"))
        joblib.dump(model_fit, model_file)

        # Predictions
        df['Predicted'] = model_fit.predict(start=0, end=len(df)-1)
        df['Predicted'] = df['Predicted'].round(2)  # round predictions

        # Rename Average Price to Actual
        df.rename(columns={'Average Price': 'Actual'}, inplace=True)

        # Calculate metrics
        y_true = df['Actual'].values
        y_pred = df['Predicted'].values
        mae = round(mean_absolute_error(y_true, y_pred), 2)
        rmse = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)
        r2 = round(r2_score(y_true, y_pred), 4)
        mape = round(mean_absolute_percentage_error_safe(y_true, y_pred), 2)
        accuracy = round(100 - mape, 2)

        print(f"Metrics for {file}:")
        print(f"  MAE        : {mae}")
        print(f"  RMSE       : {rmse}")
        print(f"  R²         : {r2}")
        print(f"  MAPE(%)    : {mape}")
        print(f"  Accuracy(%) : {accuracy}\n")

        # Save metrics
        metrics_list.append({
            'File': file,
            'MAE': mae,
            'RMSE': rmse,
            'R2': r2,
            'MAPE(%)': mape,
            'Accuracy(%)': accuracy
        })

        # -----------------------------
        # Save updated CSV with only Date, Actual, Predicted
        # -----------------------------
        save_df = df[['Actual', 'Predicted']].copy()
        save_df['Date'] = df.index
        save_df = save_df[['Date', 'Actual', 'Predicted']]
        updated_csv_path = os.path.join(output_csv, file.replace(".csv", "_sarimax_updated.csv"))
        save_df.to_csv(updated_csv_path, index=False)
        print(f"Saved updated CSV with Date, Actual & Predicted: {updated_csv_path}")

        # -----------------------------
        # Plot graph
        # -----------------------------
        plt.figure(figsize=(12,6))
        plt.plot(df.index, df['Actual'], label="Actual", color="blue")
        plt.plot(df.index, df['Predicted'], label="Predicted", color="red", linestyle="dashed")
        plt.plot(df.index, df['MA_7'], label="MA 7", color="orange")
        plt.plot(df.index, df['MA_30'], label="MA 30", color="green")
        plt.xlabel("Date")
        plt.ylabel("Average Price")
        plt.title(f"Price Prediction (SARIMAX) - {file}")
        plt.legend()
        plt.grid(True)
        graph_file = os.path.join(output_graphs, file.replace(".csv", "_sarimax_graph.png"))
        plt.savefig(graph_file, bbox_inches='tight')
        plt.close()

# Save all metrics to CSV
metrics_df = pd.DataFrame(metrics_list)
metrics_df.to_csv(output_metrics_csv, index=False)
print(f"✅ Multiple CSVs processed! Metrics saved to {output_metrics_csv}. Models, updated CSVs, and graphs saved in respective folders.")

========== Processing: onion_Bagalkot_daily.csv ==========
Metrics for onion_Bagalkot_daily.csv:
  MAE        : 117.45
  RMSE       : 200.26
  R²         : 0.9312
  MAPE(%)    : 11.52
  Accuracy(%) : 88.48

Saved updated CSV with Date, Actual & Predicted: sarimax_output_csv\onion_Bagalkot_daily_sarimax_updated.csv
========== Processing: onion_Bangalore_daily.csv ==========
Metrics for onion_Bangalore_daily.csv:
  MAE        : 365.72
  RMSE       : 547.26
  R²         : 0.7107
  MAPE(%)    : 23.43
  Accuracy(%) : 76.57

Saved updated CSV with Date, Actual & Predicted: sarimax_output_csv\onion_Bangalore_daily_sarimax_updated.csv
========== Processing: onion_Belgaum_daily.csv ==========
Metrics for onion_Belgaum_daily.csv:
  MAE        : 119.72
  RMSE       : 233.89
  R²         : 0.9439
  MAPE(%)    : 11.41
  Accuracy(%) : 88.59

Saved updated CSV with Date, Actual & Predicted: sarimax_output_csv\onion_Belgaum_daily_sarimax_updated.csv
========== Processing: onion_Bellary_daily.csv =====

In [ ]:
#TAT+MHA

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
import joblib  # For saving models as .pkl

# -----------------------------
# Output directories
# -----------------------------
input_folder = "dataset"
output_models = "tat_output_models"
output_csv = "tat_output_csv"
output_graphs = "tat_output_graphs"
output_logs = "tat_output_logs"
metrics_file = "tat_metrics.csv"

os.makedirs(output_models, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)
os.makedirs(output_logs, exist_ok=True)

# -----------------------------
# Function to create dataset
# -----------------------------
def create_dataset(data, look_back=30):
    X, y = [], []
    for i in range(len(data) - look_back):
        X.append(data[i:i+look_back, 0])
        y.append(data[i+look_back, 0])
    return np.array(X), np.array(y)

# -----------------------------
# Define Temporal Attention Transformer Model
# -----------------------------
def build_tat_model(input_shape, d_model=64, num_heads=4, ff_dim=128, dropout_rate=0.2):
    inputs = layers.Input(shape=input_shape)  # (look_back, 1)
    
    # Multi-Head Attention
    x = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)(inputs, inputs)
    x = layers.Dropout(dropout_rate)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x + inputs)  # Residual connection
    
    # Feed-Forward Network
    ff = layers.Dense(ff_dim, activation='relu')(x)
    ff = layers.Dense(input_shape[1])(ff)
    x = layers.LayerNormalization(epsilon=1e-6)(x + ff)  # Residual connection
    
    # Global Pooling
    x = layers.GlobalAveragePooling1D()(x)
    
    # Output
    outputs = layers.Dense(1)(x)
    
    model = models.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
                  loss='mse', metrics=['mae'])
    return model

# -----------------------------
# Function to robustly parse dates
# -----------------------------
def parse_dates_safe(date_series):
    # First try strict ISO format (avoids warning)
    try:
        return pd.to_datetime(date_series, format="%Y-%m-%d", errors='coerce')
    except:
        # Fallback flexible parsing
        return pd.to_datetime(date_series, errors='coerce', dayfirst=True)

# -----------------------------
# Metrics storage
# -----------------------------
metrics_list = []

# -----------------------------
# Process each CSV file
# -----------------------------
look_back = 30
for file in os.listdir(input_folder):
    if file.endswith(".csv"):
        file_path = os.path.join(input_folder, file)
        print(f"🚀 Processing: {file}")

        # Load CSV
        df = pd.read_csv(file_path)
        
        # Robust date parsing
        df['Date'] = parse_dates_safe(df['Date'])
        df = df.dropna(subset=['Date'])
        df = df.sort_values('Date')

        # Replace NaN in 'Average Price' with column mean (future-proof)
        df['Average Price'] = df['Average Price'].fillna(df['Average Price'].mean())
        df['Average Price'] = df['Average Price'].round(2)

        # Add moving averages (future-proof fill)
        df['MA_7'] = df['Average Price'].rolling(window=7).mean()
        df['MA_30'] = df['Average Price'].rolling(window=30).mean()
        df['MA_7'] = df['MA_7'].fillna(df['MA_7'].mean())
        df['MA_30'] = df['MA_30'].fillna(df['MA_30'].mean())

        # Prepare data using MinMaxScaler
        values = df[['Average Price']].values.astype('float32')
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_values = scaler.fit_transform(values)

        # Create sequences
        X, y = create_dataset(scaled_values, look_back)
        X = np.reshape(X, (X.shape[0], X.shape[1], 1))

        # Build model
        model = build_tat_model(input_shape=(look_back,1))

        # Train model
        history = model.fit(X, y, epochs=100, batch_size=16, validation_split=0.2, verbose=1)

        # Save training logs
        log_file = os.path.join(output_logs, file.replace(".csv", "_tat_training.txt"))
        with open(log_file, "w") as f:
            f.write("Training Loss per Epoch:\n")
            for i, loss in enumerate(history.history['loss']):
                f.write(f"Epoch {i+1}: Loss={loss}, Val_Loss={history.history['val_loss'][i]}\n")

        # Predictions
        predictions = model.predict(X)
        predictions_rescaled = scaler.inverse_transform(predictions)
        df['Predicted'] = [np.nan]*look_back + list(predictions_rescaled.flatten())
        df['Predicted'] = df['Predicted'].round(2)

        # Compute metrics safely
        y_true = df['Average Price'].values[look_back:]
        y_pred = predictions_rescaled.flatten()

        # Avoid divide-by-zero in MAPE
        non_zero_idx = y_true != 0
        if np.any(non_zero_idx):
            mape = round(np.mean(np.abs((y_true[non_zero_idx] - y_pred[non_zero_idx]) / y_true[non_zero_idx])) * 100, 2)
            accuracy = round(100 - mape, 2)
        else:
            mape, accuracy = np.nan, np.nan

        mae = round(mean_absolute_error(y_true, y_pred), 2)
        rmse = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)
        r2 = round(r2_score(y_true, y_pred), 4)

        metrics_list.append([file.replace(".csv",""), mae, rmse, r2, mape, accuracy])

        # Save model as .pkl
        model_file = os.path.join(output_models, file.replace(".csv", "_tat_model.pkl"))
        joblib.dump(model, model_file)

        # Save CSV with only Date, Actual, Predicted
        save_df = df[['Average Price', 'Predicted']].copy()
        save_df.rename(columns={'Average Price':'Actual'}, inplace=True)
        save_df['Date'] = df['Date']
        save_df = save_df[['Date', 'Actual', 'Predicted']]
        updated_csv_path = os.path.join(output_csv, file.replace(".csv", "_tat_updated.csv"))
        save_df.to_csv(updated_csv_path, index=False)

        # Save graph
        plt.figure(figsize=(12,6))
        plt.plot(df['Date'], df['Average Price'], label='Actual', color='blue')
        plt.plot(df['Date'], df['Predicted'], label='Predicted', color='red', linestyle='dashed')
        plt.plot(df['Date'], df['MA_7'], label='MA 7', color='orange')
        plt.plot(df['Date'], df['MA_30'], label='MA 30', color='green')
        plt.xlabel('Date')
        plt.ylabel('Average Price')
        plt.title(f'Price Prediction (TAT) - {file}')
        plt.legend()
        plt.grid(True)
        graph_file = os.path.join(output_graphs, file.replace(".csv", "_tat_graph.png"))
        plt.savefig(graph_file, bbox_inches='tight')
        plt.close()

        print(f"✅ Done with {file} | MAE={mae}, RMSE={rmse}, R2={r2}, MAPE={mape}%, Accuracy={accuracy}%\n")

# Save all metrics
metrics_df = pd.DataFrame(metrics_list, columns=['District', 'MAE', 'RMSE', 'R2', 'MAPE(%)', 'Accuracy(%)'])
metrics_df.to_csv(metrics_file, index=False)
print(f"📊 Metrics saved to {metrics_file}")

🚀 Processing: onion_Bagalkot_daily.csv
Epoch 1/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 15s 14ms/step - loss: 0.0122 - mae: 0.0736 - val_loss: 0.0111 - val_mae: 0.0686
Epoch 2/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - loss: 0.0095 - mae: 0.0676 - val_loss: 0.0107 - val_mae: 0.0680
Epoch 3/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - loss: 0.0091 - mae: 0.0669 - val_loss: 0.0109 - val_mae: 0.0684
Epoch 4/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - loss: 0.0092 - mae: 0.0662 - val_loss: 0.0105 - val_mae: 0.0679
Epoch 5/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - loss: 0.0093 - mae: 0.0664 - val_loss: 0.0105 - val_mae: 0.0679
Epoch 6/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - loss: 0.0099 - mae: 0.0689 - val_loss: 0.0104 - val_mae: 0.0678
Epoch 7/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - loss: 0.0094 - mae: 0.0675 - val_loss: 0.0104 - val_mae: 0.0678
Epoch 8/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - loss: 0.0089 - mae: 0.0655 - val_loss: 0.0097 - val_mae: 0.0677


In [ ]:
TAT+MQA

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
import joblib  # for .pkl model saving (best-effort)
warnings_imported = False
try:
    import warnings
    warnings.filterwarnings("ignore")
    warnings_imported = True
except:
    pass

# -----------------------------
# Output directories
# -----------------------------
input_folder = "dataset"
output_models = "tat_mqa_output_models"
output_csv = "tat_mqa_output_csv"
output_graphs = "tat_mqa_output_graphs"
output_logs = "tat_mqa_output_logs"
metrics_file = "tat_mqa_metrics.csv"

os.makedirs(output_models, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)
os.makedirs(output_logs, exist_ok=True)

# -----------------------------
# Function to create dataset
# -----------------------------
def create_dataset(data, look_back=30):
    X, y = [], []
    for i in range(len(data) - look_back):
        X.append(data[i:i+look_back, 0])
        y.append(data[i+look_back, 0])
    return np.array(X), np.array(y)

# -----------------------------
# Multi-Query Attention Layer
# -----------------------------
class MultiQueryAttention(layers.Layer):
    def __init__(self, num_heads, key_dim, dropout_rate=0.1, **kwargs):
        super(MultiQueryAttention, self).__init__(**kwargs)
        self.num_heads = num_heads
        self.key_dim = key_dim
        self.dropout_rate = dropout_rate
        self.q_dense = [layers.Dense(key_dim) for _ in range(num_heads)]
        self.k_dense = layers.Dense(key_dim)
        self.v_dense = layers.Dense(key_dim)
        self.dropout = layers.Dropout(dropout_rate)
        self.output_dense = layers.Dense(key_dim)

    def call(self, x):
        K = self.k_dense(x)
        V = self.v_dense(x)
        head_outputs = []

        for q_layer in self.q_dense:
            Q = q_layer(x)
            scores = tf.matmul(Q, K, transpose_b=True) / tf.math.sqrt(tf.cast(self.key_dim, tf.float32))
            attention_weights = tf.nn.softmax(scores, axis=-1)
            attention_output = tf.matmul(attention_weights, V)
            head_outputs.append(attention_output)

        concat = tf.concat(head_outputs, axis=-1)
        output = self.output_dense(concat)
        output = self.dropout(output)
        return output

    def get_config(self):
        config = super(MultiQueryAttention, self).get_config()
        config.update({
            "num_heads": self.num_heads,
            "key_dim": self.key_dim,
            "dropout_rate": self.dropout_rate
        })
        return config

# -----------------------------
# TAT-MQA Model
# -----------------------------
def build_tat_mqa_model(input_shape, d_model=64, num_heads=4, ff_dim=128, dropout_rate=0.2):
    inputs = layers.Input(shape=input_shape)
    x = MultiQueryAttention(num_heads=num_heads, key_dim=d_model, dropout_rate=dropout_rate)(inputs)
    x = layers.LayerNormalization(epsilon=1e-6)(x + inputs)

    ff = layers.Dense(ff_dim, activation='relu')(x)
    ff = layers.Dense(input_shape[1])(ff)
    x = layers.LayerNormalization(epsilon=1e-6)(x + ff)

    x = layers.GlobalAveragePooling1D()(x)
    outputs = layers.Dense(1)(x)

    model = models.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
                  loss='mse', metrics=['mae'])
    return model

# -----------------------------
# Function to robustly parse dates
# -----------------------------
def parse_dates_safe(date_series):
    # keep same behavior as you used before: infer_datetime_format True fallback
    try:
        return pd.to_datetime(date_series, infer_datetime_format=True, errors='coerce')
    except:
        return pd.to_datetime(date_series, errors='coerce')

# -----------------------------
# Metrics storage
# -----------------------------
metrics_list = []

# -----------------------------
# Process each CSV file
# -----------------------------
look_back = 30
for file in os.listdir(input_folder):
    if not file.endswith(".csv"):
        continue

    file_path = os.path.join(input_folder, file)
    print(f"🚀 Processing: {file}")

    # Load CSV
    df = pd.read_csv(file_path)

    # Robust date parsing
    df['Date'] = parse_dates_safe(df['Date'])
    df = df.dropna(subset=['Date'])
    df = df.sort_values('Date').reset_index(drop=True)

    # -----------------------------
    # Handle missing Average Price
    # -----------------------------
    if 'Average Price' not in df.columns:
        raise KeyError("Input CSV must contain 'Average Price' column.")
    if df['Average Price'].isna().sum() > 0:
        mean_val = df['Average Price'].mean()
        df['Average Price'].fillna(mean_val, inplace=True)
        print(f"Filled {df['Average Price'].isna().sum()} missing Average Price values with mean {mean_val:.2f}")

    # Round Actual values early
    df['Average Price'] = df['Average Price'].astype(float).round(2)

    # Moving averages (fill remaining NaNs with column mean to avoid plotting gaps)
    df['MA_7'] = df['Average Price'].rolling(window=7).mean()
    df['MA_30'] = df['Average Price'].rolling(window=30).mean()
    df['MA_7'].fillna(df['MA_7'].mean(), inplace=True)
    df['MA_30'].fillna(df['MA_30'].mean(), inplace=True)

    # Prepare data
    values = df[['Average Price']].values.astype('float32')
    if len(values) <= look_back:
        # Not enough data for the chosen look_back; skip file with a warning
        print(f"⚠️ Skipping {file} — dataset length ({len(values)}) <= look_back ({look_back})")
        continue

    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_values = scaler.fit_transform(values)

    X, y = create_dataset(scaled_values, look_back)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))

    # Build and train model
    model = build_tat_mqa_model(input_shape=(look_back,1))
    model.summary()

    history = model.fit(X, y, epochs=100, batch_size=16, validation_split=0.2, verbose=1)

    # Save training logs
    log_file = os.path.join(output_logs, file.replace(".csv", "_tat_mqa_training.txt"))
    with open(log_file, "w") as f:
        f.write("Training Loss per Epoch:\n")
        for i, loss in enumerate(history.history['loss']):
            val_loss = history.history['val_loss'][i] if 'val_loss' in history.history else None
            f.write(f"Epoch {i+1}: Loss={loss}, Val_Loss={val_loss}\n")

    # Predictions (rescale)
    predictions = model.predict(X)
    predictions_rescaled = scaler.inverse_transform(predictions).flatten()
    # pad with NaNs at start so predicted series aligns with original df index
    padded_preds = np.concatenate([np.full(look_back, np.nan), predictions_rescaled])
    df['Predicted'] = np.round(padded_preds, 2)

    # -----------------------------
    # Compute metrics using only the matched portion (exclude leading NaNs)
    # -----------------------------
    y_true = df['Average Price'].values[look_back:]
    y_pred = predictions_rescaled
    mae = round(mean_absolute_error(y_true, y_pred), 2)
    rmse = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)
    r2 = round(r2_score(y_true, y_pred), 4)
    # MAPE: avoid divide-by-zero by masking zeros
    mask = y_true != 0
    if mask.sum() == 0:
        mape = np.nan
        accuracy = np.nan
    else:
        mape = round(np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100, 2)
        accuracy = round(100 - mape, 2)

    metrics_list.append([file.replace(".csv",""), mae, rmse, r2, mape, accuracy])

    print(f"✅ Done with {file} | MAE={mae}, RMSE={rmse}, R2={r2}, MAPE={mape}%, Accuracy={accuracy}%")

    # -----------------------------
    # Save model: native Keras and attempt .pkl (best-effort)
    # -----------------------------
    keras_model_path = os.path.join(output_models, file.replace(".csv", "_tat_mqa_model.keras"))
    model.save(keras_model_path)
    pkl_model_path = os.path.join(output_models, file.replace(".csv", "_tat_mqa_model.pkl"))
    try:
        # joblib.dump the Keras model object (may fail on some TF versions)
        joblib.dump(model, pkl_model_path)
    except Exception as e:
        # fallback: save model architecture + weights dict to pickle (best-effort)
        try:
            model_info = {
                "config": model.get_config(),
                "weights": model.get_weights()
            }
            joblib.dump(model_info, pkl_model_path)
        except Exception as e2:
            print(f"⚠️ Could not save .pkl for {file}: {e}; {e2}")

    # -----------------------------
    # Save CSV with only Date, Actual, Predicted (Actual renamed)
    # -----------------------------
    save_df = pd.DataFrame({
        'Date': df['Date'],
        'Actual': df['Average Price'].round(2),
        'Predicted': df['Predicted']
    })
    updated_csv_path = os.path.join(output_csv, file.replace(".csv", "_tat_mqa_updated.csv"))
    save_df.to_csv(updated_csv_path, index=False)
    print(f"Saved updated CSV with Date, Actual & Predicted: {updated_csv_path}")

    # -----------------------------
    # Save graph with Actual, Predicted, MA7, MA30
    # -----------------------------
    plt.figure(figsize=(12,6))
    plt.plot(df['Date'], df['Average Price'], label='Actual', color='blue')
    plt.plot(df['Date'], df['Predicted'], label='Predicted', color='red', linestyle='dashed')
    plt.plot(df['Date'], df['MA_7'], label='MA 7', color='orange')
    plt.plot(df['Date'], df['MA_30'], label='MA 30', color='green')
    plt.xlabel('Date')
    plt.ylabel('Average Price')
    plt.title(f'Price Prediction (TAT-MQA) - {file}')
    plt.legend()
    plt.grid(True)
    graph_file = os.path.join(output_graphs, file.replace(".csv", "_tat_mqa_graph.png"))
    plt.savefig(graph_file, bbox_inches='tight')
    plt.close()

# -----------------------------
# Save all metrics
# -----------------------------
metrics_df = pd.DataFrame(metrics_list, columns=['District', 'MAE', 'RMSE', 'R2', 'MAPE(%)', 'Accuracy(%)'])
metrics_df.to_csv(metrics_file, index=False)
print(f"📊 Metrics saved to {metrics_file}")

In [ ]:
#TAT + GQA

In [ ]:
import os
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
import joblib  # For saving models as .pkl

# -----------------------------
# Suppress Warnings
# -----------------------------
warnings.filterwarnings("ignore", category=FutureWarning)
pd.options.mode.chained_assignment = None  # Avoid SettingWithCopyWarning

# -----------------------------
# Output directories
# -----------------------------
input_folder = "dataset"
output_models = "tat_gqa_output_models"
output_csv = "tat_gqa_output_csv"
output_graphs = "tat_gqa_output_graphs"
output_logs = "tat_gqa_output_logs"
metrics_file = "tat_gqa_metrics.csv"

for folder in [output_models, output_csv, output_graphs, output_logs]:
    os.makedirs(folder, exist_ok=True)

# -----------------------------
# Function to create dataset
# -----------------------------
def create_dataset(data, look_back=30):
    X, y = [], []
    for i in range(len(data) - look_back):
        X.append(data[i:i + look_back, 0])
        y.append(data[i + look_back, 0])
    return np.array(X), np.array(y)

# -----------------------------
# Grouped Query Attention Layer
# -----------------------------
class GroupedQueryAttention(layers.Layer):
    def __init__(self, num_groups=2, key_dim=64, dropout_rate=0.1, **kwargs):
        super(GroupedQueryAttention, self).__init__(**kwargs)
        self.num_groups = num_groups
        self.key_dim = key_dim
        self.dropout_rate = dropout_rate
        self.k_dense = layers.Dense(key_dim)
        self.v_dense = layers.Dense(key_dim)
        self.q_dense_groups = [layers.Dense(key_dim) for _ in range(num_groups)]
        self.dropout = layers.Dropout(dropout_rate)
        self.output_dense = layers.Dense(key_dim)

    def call(self, x):
        K = self.k_dense(x)
        V = self.v_dense(x)
        group_outputs = []
        for q_layer in self.q_dense_groups:
            Q = q_layer(x)
            scores = tf.matmul(Q, K, transpose_b=True) / tf.math.sqrt(tf.cast(self.key_dim, tf.float32))
            attention_weights = tf.nn.softmax(scores, axis=-1)
            attn_output = tf.matmul(attention_weights, V)
            group_outputs.append(attn_output)
        concat = tf.concat(group_outputs, axis=-1)
        output = self.output_dense(concat)
        output = self.dropout(output)
        return output

    def get_config(self):
        config = super(GroupedQueryAttention, self).get_config()
        config.update({
            "num_groups": self.num_groups,
            "key_dim": self.key_dim,
            "dropout_rate": self.dropout_rate
        })
        return config

# -----------------------------
# TAT-GQA Model
# -----------------------------
def build_tat_gqa_model(input_shape, d_model=64, num_groups=2, ff_dim=128, dropout_rate=0.2):
    inputs = layers.Input(shape=input_shape)
    x = GroupedQueryAttention(num_groups=num_groups, key_dim=d_model, dropout_rate=dropout_rate)(inputs)
    x = layers.LayerNormalization(epsilon=1e-6)(x + inputs)
    ff = layers.Dense(ff_dim, activation='relu')(x)
    ff = layers.Dense(input_shape[1])(ff)
    x = layers.LayerNormalization(epsilon=1e-6)(x + ff)
    x = layers.GlobalAveragePooling1D()(x)
    outputs = layers.Dense(1)(x)
    model = models.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
                  loss='mse', metrics=['mae'])
    return model

# -----------------------------
# Metrics storage
# -----------------------------
metrics_list = []

# -----------------------------
# Training Configuration
# -----------------------------
look_back = 30
epochs = 100
batch_size = 16

# -----------------------------
# Process each CSV file
# -----------------------------
for file in os.listdir(input_folder):
    if not file.endswith(".csv"):
        continue

    file_path = os.path.join(input_folder, file)
    print(f"🚀 Processing: {file}")

    # Load CSV
    try:
        df = pd.read_csv(file_path)
    except Exception as e:
        print(f"❌ Failed to read {file}: {e}")
        continue

    # Parse dates safely (no infer_datetime_format)
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    df = df.dropna(subset=['Date']).sort_values('Date')

    # Check if column exists
    if 'Average Price' not in df.columns:
        print(f"⚠️ Skipping {file}: 'Average Price' column not found.")
        continue

    # Handle missing price values
    df['Average Price'] = df['Average Price'].fillna(df['Average Price'].mean())

    # Moving averages (for visualization only)
    df['MA_7'] = df['Average Price'].rolling(window=7).mean().fillna(df['Average Price'].mean())
    df['MA_30'] = df['Average Price'].rolling(window=30).mean().fillna(df['Average Price'].mean())

    # Prepare scaled data
    values = df[['Average Price']].astype('float32').values
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_values = scaler.fit_transform(values)
    X, y = create_dataset(scaled_values, look_back)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))

    # Build and train model
    model = build_tat_gqa_model(input_shape=(look_back, 1))
    history = model.fit(
        X, y,
        epochs=epochs,
        batch_size=batch_size,
        validation_split=0.2,
        verbose=1
    )

    # Save training log
    log_file = os.path.join(output_logs, file.replace(".csv", "_tat_gqa_training.txt"))
    with open(log_file, "w") as f:
        f.write("Epoch\tLoss\tVal_Loss\n")
        for i in range(len(history.history['loss'])):
            f.write(f"{i+1}\t{history.history['loss'][i]:.6f}\t{history.history['val_loss'][i]:.6f}\n")

    # Predictions
    predictions = model.predict(X)
    predictions_rescaled = scaler.inverse_transform(predictions)
    df['Predicted'] = [np.nan] * look_back + list(predictions_rescaled.flatten())

    # Round values for clean output
    df['Average Price'] = df['Average Price'].round(2)
    df['Predicted'] = df['Predicted'].round(2)

    # Metrics Calculation
    y_true = df['Average Price'].values[look_back:]
    y_pred = predictions_rescaled.flatten()

    # Avoid division by zero or NaN in MAPE
    mask = y_true != 0
    if np.any(mask):
        mape = np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100
    else:
        mape = 0.0

    mae = round(mean_absolute_error(y_true, y_pred), 2)
    rmse = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)
    r2 = round(r2_score(y_true, y_pred), 2)
    mape = round(mape, 2)
    accuracy = round(100 - mape, 2)

    metrics_list.append([file.replace(".csv", ""), mae, rmse, r2, mape, accuracy])

    # Save model as .pkl
    model_file = os.path.join(output_models, file.replace(".csv", "_tat_gqa_model.pkl"))
    joblib.dump(model, model_file)

    # Save predictions to CSV
    save_df = df[['Date', 'Average Price', 'Predicted']].rename(columns={'Average Price': 'Actual'})
    updated_csv_path = os.path.join(output_csv, file.replace(".csv", "_tat_gqa_updated.csv"))
    save_df.to_csv(updated_csv_path, index=False)

    # Plot Graph
    plt.figure(figsize=(12, 6))
    plt.plot(df['Date'], df['Average Price'], label='Actual', color='blue')
    plt.plot(df['Date'], df['MA_7'], label='MA 7', color='orange')
    plt.plot(df['Date'], df['MA_30'], label='MA 30', color='green')
    plt.plot(df['Date'], df['Predicted'], label='Predicted (TAT-GQA)', color='red', linestyle='dashed')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.title(f'Price Prediction (TAT-GQA) - {file}')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    graph_file = os.path.join(output_graphs, file.replace(".csv", "_tat_gqa_graph.png"))
    plt.savefig(graph_file)
    plt.close()

    print(f"✅ Done: {file} | MAE={mae}, RMSE={rmse}, R2={r2}, MAPE={mape}%, Accuracy={accuracy}%")

# -----------------------------
# Save Metrics
# -----------------------------
metrics_df = pd.DataFrame(metrics_list, columns=['District', 'MAE', 'RMSE', 'R2', 'MAPE(%)', 'Accuracy(%)'])
metrics_df.to_csv(metrics_file, index=False)
print(f"📊 All metrics saved to: {metrics_file}")

🚀 Processing: onion_Bagalkot_daily.csv
Epoch 1/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - loss: 0.1170 - mae: 0.1838 - val_loss: 0.0042 - val_mae: 0.0567
Epoch 2/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0093 - mae: 0.0705 - val_loss: 0.0023 - val_mae: 0.0392
Epoch 3/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0061 - mae: 0.0582 - val_loss: 0.0035 - val_mae: 0.0397
Epoch 4/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0044 - mae: 0.0475 - val_loss: 0.0017 - val_mae: 0.0279
Epoch 5/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0036 - mae: 0.0412 - val_loss: 0.0021 - val_mae: 0.0357
Epoch 6/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0026 - mae: 0.0357 - val_loss: 0.0017 - val_mae: 0.0289
Epoch 7/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0027 - mae: 0.0354 - val_loss: 0.0020 - val_mae: 0.0350
Epoch 8/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0023 - mae: 0.0337 - val_loss: 0.0028 - val_mae: 0.0413
Epoch 9/1

In [ ]:
#TAT+HA

In [3]:
import os
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers

# =========================================================
# CLEAN STARTUP – suppress all unnecessary warnings/logs
# =========================================================
warnings.filterwarnings("ignore")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Disable TensorFlow INFO/WARNING logs
tf.get_logger().setLevel("ERROR")

# =========================================================
# Output directories
# =========================================================
input_folder = "dataset"
output_models = "tat_ha_output_models"
output_csv = "tat_ha_output_csv"
output_graphs = "tat_ha_output_graphs"
output_logs = "tat_ha_output_logs"
metrics_file = "tat_ha_metrics.csv"

os.makedirs(output_models, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)
os.makedirs(output_logs, exist_ok=True)

# =========================================================
# Dataset creation
# =========================================================
def create_dataset(data, look_back=30):
    X, y = [], []
    for i in range(len(data) - look_back):
        X.append(data[i:i + look_back, 0])
        y.append(data[i + look_back, 0])
    return np.array(X), np.array(y)

# =========================================================
# Local Attention Layer
# =========================================================
class LocalAttention(layers.Layer):
    def __init__(self, key_dim=64, dropout_rate=0.1, **kwargs):
        super(LocalAttention, self).__init__(**kwargs)
        self.key_dim = key_dim
        self.dropout_rate = dropout_rate
        self.q_dense = layers.Dense(key_dim)
        self.k_dense = layers.Dense(key_dim)
        self.v_dense = layers.Dense(key_dim)
        self.dropout = layers.Dropout(dropout_rate)

    def call(self, x):
        Q = self.q_dense(x)
        K = self.k_dense(x)
        V = self.v_dense(x)
        scores = tf.matmul(Q, K, transpose_b=True) / tf.math.sqrt(
            tf.cast(tf.shape(K)[-1], tf.float32))
        weights = tf.nn.softmax(scores, axis=-1)
        output = tf.matmul(weights, V)
        return self.dropout(output)

# =========================================================
# Hierarchical Attention Model
# =========================================================
def build_tat_ha_model(input_shape, d_model=64, ff_dim=128, dropout_rate=0.2):
    inputs = layers.Input(shape=input_shape)
    local_attn = LocalAttention(key_dim=d_model, dropout_rate=dropout_rate)(inputs)
    local_attn = layers.LayerNormalization(epsilon=1e-6)(local_attn + inputs)

    global_attn = layers.MultiHeadAttention(num_heads=4, key_dim=d_model)(local_attn, local_attn)
    global_attn = layers.Dropout(dropout_rate)(global_attn)
    global_attn = layers.LayerNormalization(epsilon=1e-6)(global_attn + local_attn)

    ff = layers.Dense(ff_dim, activation='relu')(global_attn)
    ff = layers.Dense(input_shape[1])(ff)
    x = layers.LayerNormalization(epsilon=1e-6)(ff + global_attn)
    x = layers.GlobalAveragePooling1D()(x)

    outputs = layers.Dense(1)(x)
    model = models.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
                  loss='mse', metrics=['mae'])
    return model

# =========================================================
# Metrics list
# =========================================================
metrics_list = []

# =========================================================
# Process each CSV file
# =========================================================
look_back = 30
for file in os.listdir(input_folder):
    if not file.endswith(".csv"):
        continue

    file_path = os.path.join(input_folder, file)
    print(f"\n🚀 Processing: {file}")

    # Load CSV
    df = pd.read_csv(file_path)

    # Safe datetime conversion
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    df = df.dropna(subset=['Date']).sort_values('Date')

    # Ensure target column exists
    if 'Average Price' not in df.columns:
        print(f"⚠️ Skipping {file}: 'Average Price' missing.")
        continue

    # Replace inplace warnings (no inplace=True used)
    df['Average Price'] = df['Average Price'].fillna(df['Average Price'].mean())
    df['MA_7'] = df['Average Price'].rolling(window=7).mean()
    df['MA_30'] = df['Average Price'].rolling(window=30).mean()
    df['MA_7'] = df['MA_7'].fillna(df['MA_7'].mean())
    df['MA_30'] = df['MA_30'].fillna(df['MA_30'].mean())

    # Prepare training data
    values = df[['Average Price']].values.astype('float32')
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_values = scaler.fit_transform(values)
    if len(scaled_values) <= look_back:
        print(f"⚠️ Skipping {file}: not enough data points.")
        continue

    X, y = create_dataset(scaled_values, look_back)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))

    # Build and train model (show epochs cleanly)
    model = build_tat_ha_model(input_shape=(look_back, 1))
    history = model.fit(X, y, epochs=50, batch_size=16, validation_split=0.2, verbose=2)

    # Save logs
    log_file = os.path.join(output_logs, file.replace(".csv", "_tat_ha_training.txt"))
    with open(log_file, "w") as f:
        f.write("Training Loss per Epoch:\n")
        for i, loss in enumerate(history.history['loss']):
            f.write(f"Epoch {i + 1}: Loss={loss}, Val_Loss={history.history['val_loss'][i]}\n")

    # Predictions
    predictions = model.predict(X, verbose=0)
    predictions_rescaled = scaler.inverse_transform(predictions)
    df['Predicted'] = [np.nan] * look_back + list(predictions_rescaled.flatten())

    # Round results
    df['Average Price'] = df['Average Price'].round(2)
    df['Predicted'] = df['Predicted'].round(2)

    # Evaluate
    y_true = df['Average Price'].values[look_back:]
    y_pred = predictions_rescaled.flatten()
    valid_idx = ~np.isnan(y_true) & ~np.isnan(y_pred) & (y_true != 0)

    if not np.any(valid_idx):
        print(f"⚠️ Skipping metrics for {file}: invalid or zero values.")
        continue

    y_true, y_pred = y_true[valid_idx], y_pred[valid_idx]
    mae = round(mean_absolute_error(y_true, y_pred), 2)
    rmse = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)
    r2 = round(r2_score(y_true, y_pred), 2)
    mape = round(np.mean(np.abs((y_true - y_pred) / np.maximum(y_true, 1e-10))) * 100, 2)
    accuracy = round(max(0, min(100, 100 - mape)), 2)

    metrics_list.append([file.replace(".csv", ""), mae, rmse, r2, mape, accuracy])

    # Save model
    model_file = os.path.join(output_models, file.replace(".csv", "_tat_ha_model.pkl"))
    joblib.dump(model, model_file)

    # Save updated CSV
    save_df = df[['Date', 'Average Price', 'Predicted']].rename(columns={'Average Price': 'Actual'})
    updated_csv_path = os.path.join(output_csv, file.replace(".csv", "_tat_ha_updated.csv"))
    save_df.to_csv(updated_csv_path, index=False)

    # Plot graph
    plt.figure(figsize=(12, 6))
    plt.plot(df['Date'], df['Average Price'], label='Actual', color='blue')
    plt.plot(df['Date'], df['MA_7'], label='MA 7', color='orange')
    plt.plot(df['Date'], df['MA_30'], label='MA 30', color='green')
    plt.plot(df['Date'], df['Predicted'], label='Predicted (TAT-HA)', color='red', linestyle='dashed')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.title(f'Price Prediction (TAT-HA) - {file}')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    graph_file = os.path.join(output_graphs, file.replace(".csv", "_tat_ha_graph.png"))
    plt.savefig(graph_file)
    plt.close()

    print(f"✅ Done with {file} | MAE={mae}, RMSE={rmse}, R2={r2}, MAPE={mape}%, Accuracy={accuracy}%")

# Save metrics
if metrics_list:
    metrics_df = pd.DataFrame(metrics_list, columns=['District', 'MAE', 'RMSE', 'R2', 'MAPE(%)', 'Accuracy(%)'])
    metrics_df.to_csv(metrics_file, index=False)
    print(f"\n📊 Metrics saved to {metrics_file}")
else:
    print("\n⚠️ No valid data found to save metrics.")

451/451 - 7s - 16ms/step - loss: 0.0017 - mae: 0.0277 - val_loss: 0.0052 - val_mae: 0.0596
Epoch 19/50
451/451 - 6s - 14ms/step - loss: 0.0017 - mae: 0.0274 - val_loss: 0.0032 - val_mae: 0.0427
Epoch 20/50
451/451 - 6s - 13ms/step - loss: 0.0018 - mae: 0.0279 - val_loss: 0.0040 - val_mae: 0.0539
Epoch 21/50
451/451 - 6s - 14ms/step - loss: 0.0017 - mae: 0.0273 - val_loss: 0.0021 - val_mae: 0.0356
Epoch 22/50
451/451 - 6s - 13ms/step - loss: 0.0017 - mae: 0.0277 - val_loss: 0.0035 - val_mae: 0.0508
Epoch 23/50
451/451 - 6s - 14ms/step - loss: 0.0017 - mae: 0.0276 - val_loss: 0.0027 - val_mae: 0.0409
Epoch 24/50
451/451 - 6s - 14ms/step - loss: 0.0017 - mae: 0.0273 - val_loss: 0.0023 - val_mae: 0.0406
Epoch 25/50
451/451 - 8s - 17ms/step - loss: 0.0016 - mae: 0.0260 - val_loss: 0.0023 - val_mae: 0.0373
Epoch 26/50
451/451 - 7s - 15ms/step - loss: 0.0017 - mae: 0.0275 - val_loss: 0.0077 - val_mae: 0.0760
Epoch 27/50
451/451 - 6s - 14ms/step - loss: 0.0017 - mae: 0.0272 - val_loss: 0.0017 

In [ ]:
#LSTM

In [4]:
import os
import warnings
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import joblib
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# -----------------------------
# 🧹 Clean console output
# -----------------------------
warnings.filterwarnings("ignore")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'   # Hide TensorFlow INFO/WARNING/ERROR
tf.get_logger().setLevel("ERROR")          # Suppress TF backend logs

# -----------------------------
# Paths
# -----------------------------
input_folder = "dataset"
output_models = "output_lstm_models"
output_csv = "output_lstm_csv"
output_graphs = "output_lstm_graphs"
output_logs = "output_lstm_logs"
metrics_file = os.path.join(output_csv, "lstm_metrics.csv")

os.makedirs(output_models, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)
os.makedirs(output_logs, exist_ok=True)

# -----------------------------
# Function to create sequences
# -----------------------------
def create_sequences(data, seq_length=5):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

# -----------------------------
# Metrics storage
# -----------------------------
metrics_list = []

# -----------------------------
# Process each CSV
# -----------------------------
seq_length = 5
for file in os.listdir(input_folder):
    if file.endswith(".csv"):
        district_name = file.split(".")[0]
        print(f"\n========== Processing: {file} ==========")

        # Load CSV with multiple date formats
        df = pd.read_csv(os.path.join(input_folder, file))
        df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
        df = df.dropna(subset=['Date'])
        df = df.sort_values('Date')

        # Fill missing Average Price
        df['Average Price'] = df['Average Price'].fillna(df['Average Price'].mean())

        # Moving averages (for plotting only)
        df['MA_7'] = df['Average Price'].rolling(window=7).mean().fillna(df['Average Price'].mean())
        df['MA_30'] = df['Average Price'].rolling(window=30).mean().fillna(df['Average Price'].mean())

        # Scaling
        scaler = MinMaxScaler()
        prices_scaled = scaler.fit_transform(df['Average Price'].values.reshape(-1, 1))

        # Prepare sequences
        X, y = create_sequences(prices_scaled, seq_length)

        # Train-test split (80%-20%)
        split = int(0.8 * len(X))
        X_train, X_test = X[:split], X[split:]
        y_train, y_test = y[:split], y[split:]

        # Build LSTM model
        model = Sequential()
        model.add(LSTM(50, activation='relu', input_shape=(seq_length, 1)))
        model.add(Dense(1))
        model.compile(optimizer='adam', loss='mse', metrics=['mae'])

        # Train model (Epochs will show in console)
        history = model.fit(
            X_train, y_train,
            epochs=20,
            batch_size=32,
            validation_data=(X_test, y_test),
            verbose=1  # 👈 keeps epoch progress in console
        )

        # Predictions
        y_pred_scaled = model.predict(X_test)
        y_pred = scaler.inverse_transform(y_pred_scaled)
        y_true = scaler.inverse_transform(y_test.reshape(-1, 1))

        # Round Actual & Predicted
        y_true_round = np.round(y_true, 2)
        y_pred_round = np.round(y_pred, 2)

        # Save predictions in CSV (only Date, Actual, Predicted)
        df_pred = df.iloc[seq_length + split:].copy()
        df_pred = df_pred[['Date']].copy()
        df_pred['Actual'] = y_true_round.flatten()
        df_pred['Predicted'] = y_pred_round.flatten()
        df_pred.to_csv(os.path.join(output_csv, f"{district_name}_lstm_updated.csv"), index=False)

        # Metrics calculation
        mae_val = round(mean_absolute_error(y_true, y_pred), 2)
        rmse_val = round(mean_squared_error(y_true, y_pred, squared=False), 2)
        r2_val = round(r2_score(y_true, y_pred), 2)
        mape_val = round(np.mean(np.abs((y_true - y_pred) / y_true)) * 100, 2)
        accuracy_val = round(100 - mape_val, 2)
        metrics_list.append([district_name, mae_val, rmse_val, r2_val, mape_val, accuracy_val])

        print(f"✅ Done with {file} | MAE={mae_val}, RMSE={rmse_val}, R2={r2_val}, "
              f"MAPE={mape_val}%, Accuracy={accuracy_val}%")

        # Save model as .pkl
        model_file = os.path.join(output_models, f"{district_name}_lstm_model.pkl")
        joblib.dump(model, model_file)

        # Save prediction graph (with MA7 & MA30 only for graphing)
        plt.figure(figsize=(12, 6))
        plt.plot(df['Date'], df['Average Price'], label='Actual', color='blue')
        plt.plot(df['Date'], df['MA_7'], label='MA 7', color='orange')
        plt.plot(df['Date'], df['MA_30'], label='MA 30', color='green')
        plt.plot(df_pred['Date'], df_pred['Predicted'], label='Predicted', color='red', linestyle='dashed')
        plt.xlabel('Date')
        plt.ylabel('Average Price')
        plt.title(f"LSTM Predictions for {district_name}")
        plt.legend()
        plt.grid(True)
        plt.savefig(os.path.join(output_graphs, f"{district_name}_lstm_graph.png"))
        plt.close()

        # Save training loss graph
        plt.figure(figsize=(8, 4))
        plt.plot(history.history['loss'], label='Train Loss')
        plt.plot(history.history['val_loss'], label='Val Loss')
        plt.title(f"Training Loss for {district_name}")
        plt.xlabel("Epochs")
        plt.ylabel("Loss")
        plt.legend()
        plt.savefig(os.path.join(output_graphs, f"{district_name}_lstm_training_loss.png"))
        plt.close()

# Save metrics CSV
metrics_df = pd.DataFrame(metrics_list, columns=['District', 'MAE', 'RMSE', 'R2', 'MAPE(%)', 'Accuracy(%)'])
metrics_df.to_csv(metrics_file, index=False)
print(f"\n📊 Metrics saved to {metrics_file}")
print("\nAll districts processed successfully!")


========== Processing: onion_Bagalkot_daily.csv ==========
Epoch 1/20
226/226 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - loss: 0.0063 - mae: 0.0467 - val_loss: 3.3103e-04 - val_mae: 0.0109
Epoch 2/20
226/226 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 8.8581e-04 - mae: 0.0193 - val_loss: 3.1775e-04 - val_mae: 0.0099
Epoch 3/20
226/226 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 9.0507e-04 - mae: 0.0195 - val_loss: 5.8035e-04 - val_mae: 0.0162
Epoch 4/20
226/226 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 9.4885e-04 - mae: 0.0200 - val_loss: 3.0536e-04 - val_mae: 0.0099
Epoch 5/20
226/226 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 8.8410e-04 - mae: 0.0193 - val_loss: 3.1434e-04 - val_mae: 0.0106
Epoch 6/20
226/226 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 8.0755e-04 - mae: 0.0188 - val_loss: 3.1791e-04 - val_mae: 0.0112
Epoch 7/20
226/226 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 8.1926e-04 - mae: 0.0185 - val_loss: 3.2336e-04 - val_mae: 0.0126
Epoch 8/20
226/226 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 

In [ ]:
#GRU

In [1]:
import os
import warnings
warnings.filterwarnings("ignore")   # 👈 Hide all warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout

# -----------------------------
# Output paths
# -----------------------------
input_folder = "dataset"
output_models = "gru_output_models"
output_csv = "gru_output_csv"
output_graphs = "gru_output_graphs"
output_logs = "gru_output_logs"
metrics_file = os.path.join(output_csv, "gru_metrics.csv")

os.makedirs(output_models, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)
os.makedirs(output_logs, exist_ok=True)

# -----------------------------
# Function to create dataset
# -----------------------------
def create_dataset(data, look_back=30):
    X, y = [], []
    for i in range(len(data) - look_back):
        X.append(data[i:i + look_back, 0])
        y.append(data[i + look_back, 0])
    return np.array(X), np.array(y)

# -----------------------------
# Metrics storage
# -----------------------------
metrics_list = []

# -----------------------------
# Process each CSV file
# -----------------------------
look_back = 30
for file in os.listdir(input_folder):
    if file.endswith(".csv"):
        district_name = file.split(".")[0]
        print(f"\n🚀 Processing: {file}")

        # Load dataset
        df = pd.read_csv(os.path.join(input_folder, file))

        # Robust date parsing
        df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
        df = df.dropna(subset=['Date'])
        df = df.sort_values('Date')

        # Handle missing values
        df['Average Price'] = df['Average Price'].fillna(df['Average Price'].mean())

        # Add moving averages
        df['MA_7'] = df['Average Price'].rolling(window=7).mean().fillna(df['Average Price'].mean())
        df['MA_30'] = df['Average Price'].rolling(window=30).mean().fillna(df['Average Price'].mean())

        # Scale data
        values = df[['Average Price']].values
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_values = scaler.fit_transform(values)

        X, y = create_dataset(scaled_values, look_back)
        X = X.reshape((X.shape[0], X.shape[1], 1))

        # Train-test split
        split = int(len(X) * 0.8)
        X_train, X_test = X[:split], X[split:]
        y_train, y_test = y[:split], y[split:]

        # Build GRU model
        model = Sequential([
            GRU(64, return_sequences=True, input_shape=(look_back, 1)),
            Dropout(0.2),
            GRU(32),
            Dropout(0.2),
            Dense(1)
        ])
        model.compile(optimizer="adam", loss="mse", metrics=['mae'])

        # Train model — EPOCHS will display in console
        history = model.fit(X_train, y_train, epochs=20, batch_size=16,
                            validation_data=(X_test, y_test), verbose=1)

        # Save training log
        log_file = os.path.join(output_logs, f"{district_name}_gru_training.txt")
        with open(log_file, "w") as f:
            f.write("Epoch\tTrain_Loss\tVal_Loss\n")
            for i, (loss, val_loss) in enumerate(zip(history.history['loss'], history.history['val_loss'])):
                f.write(f"{i+1}\t{loss:.6f}\t{val_loss:.6f}\n")

        # Predictions
        y_pred_scaled = model.predict(X_test)
        y_pred = scaler.inverse_transform(y_pred_scaled)
        y_true = scaler.inverse_transform(y_test.reshape(-1, 1))

        # Round predictions
        y_true_round = np.round(y_true, 2)
        y_pred_round = np.round(y_pred, 2)

        # Save predictions CSV (only Date, Actual, Predicted)
        df_pred = df.iloc[-len(y_true):].copy()
        df_pred = df_pred[['Date']].reset_index(drop=True)
        df_pred['Actual'] = y_true_round.flatten()
        df_pred['Predicted'] = y_pred_round.flatten()
        df_pred.to_csv(os.path.join(output_csv, f"{district_name}_gru_updated.csv"), index=False)

        # Metrics
        mae_val = round(mean_absolute_error(y_true, y_pred), 2)
        rmse_val = round(mean_squared_error(y_true, y_pred, squared=False), 2)
        r2_val = round(r2_score(y_true, y_pred), 2)
        mape_val = round(np.mean(np.abs((y_true - y_pred) / y_true)) * 100, 2)
        accuracy_val = round(100 - mape_val, 2)
        metrics_list.append([district_name, mae_val, rmse_val, r2_val, mape_val, accuracy_val])

        print(f"✅ Done with {file} | MAE={mae_val}, RMSE={rmse_val}, R2={r2_val}, "
              f"MAPE={mape_val}%, Accuracy={accuracy_val}%")

        # Save model as .pkl
        model_file = os.path.join(output_models, f"{district_name}_gru_model.pkl")
        joblib.dump(model, model_file)

        # Save prediction graph (with MA7 & MA30)
        plt.figure(figsize=(12, 6))
        plt.plot(df['Date'], df['Average Price'], label="Actual", color="blue")
        plt.plot(df['Date'], df['MA_7'], label="MA_7", color="orange")
        plt.plot(df['Date'], df['MA_30'], label="MA_30", color="green")
        plt.plot(df_pred['Date'], df_pred['Predicted'], label="Predicted (GRU)", color="red", linestyle="dashed")
        plt.xlabel("Date")
        plt.ylabel("Average Price")
        plt.title(f"GRU Predictions - {district_name}")
        plt.legend()
        plt.grid(True)
        plt.savefig(os.path.join(output_graphs, f"{district_name}_gru_graph.png"))
        plt.close()

        # Training loss graph
        plt.figure(figsize=(8, 4))
        plt.plot(history.history['loss'], label='Train Loss')
        plt.plot(history.history['val_loss'], label='Val Loss')
        plt.title(f"GRU Training Loss - {district_name}")
        plt.xlabel("Epochs")
        plt.ylabel("Loss")
        plt.legend()
        plt.savefig(os.path.join(output_graphs, f"{district_name}_gru_loss.png"))
        plt.close()

# Save metrics CSV
metrics_df = pd.DataFrame(metrics_list, columns=['District', 'MAE', 'RMSE', 'R2', 'MAPE(%)', 'Accuracy(%)'])
metrics_df.to_csv(metrics_file, index=False)
print("\n📊 Metrics saved to", metrics_file)
print("\n✅ All districts processed successfully!")


🚀 Processing: onion_Bagalkot_daily.csv
Epoch 1/20
451/451 ━━━━━━━━━━━━━━━━━━━━ 13s 18ms/step - loss: 0.0031 - mae: 0.0340 - val_loss: 8.0192e-04 - val_mae: 0.0214
Epoch 2/20
451/451 ━━━━━━━━━━━━━━━━━━━━ 8s 18ms/step - loss: 0.0011 - mae: 0.0225 - val_loss: 4.4707e-04 - val_mae: 0.0123
Epoch 3/20
451/451 ━━━━━━━━━━━━━━━━━━━━ 9s 19ms/step - loss: 0.0011 - mae: 0.0224 - val_loss: 3.7564e-04 - val_mae: 0.0118
Epoch 4/20
451/451 ━━━━━━━━━━━━━━━━━━━━ 8s 17ms/step - loss: 0.0011 - mae: 0.0220 - val_loss: 3.4057e-04 - val_mae: 0.0121
Epoch 5/20
451/451 ━━━━━━━━━━━━━━━━━━━━ 8s 18ms/step - loss: 0.0010 - mae: 0.0208 - val_loss: 3.5499e-04 - val_mae: 0.0148
Epoch 6/20
451/451 ━━━━━━━━━━━━━━━━━━━━ 8s 17ms/step - loss: 8.8227e-04 - mae: 0.0195 - val_loss: 4.0540e-04 - val_mae: 0.0147
Epoch 7/20
451/451 ━━━━━━━━━━━━━━━━━━━━ 8s 17ms/step - loss: 9.6367e-04 - mae: 0.0201 - val_loss: 2.6771e-04 - val_mae: 0.0100
Epoch 8/20
451/451 ━━━━━━━━━━━━━━━━━━━━ 8s 17ms/step - loss: 8.6980e-04 - mae: 0.0198 - va